# CHAPTER 7 Data Cleaning and Preparation

其实数据分析中80%的时间都是在数据清理部分，loading, clearning, transforming, rearranging。而pandas非常适合用来执行这些任务。

In [1]:
import pandas as pd
import numpy as np

## 7.1 Handling Missing Data

在pandas中，missing data呈现的方式有些缺点的，但对大部分用户能起到足够的效果。对于数值型数据，pandas用浮点数Nan(Not a Number)来表示缺失值。我们称之为识别符（sentinel value)，这种值能被轻易检测到：|

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

在pandas中，我们使用了R语言中的一些传统，把缺失值表示为NA（not available）。在统计应用里，NA数据要么是数据不存在，要么是存在但不能被检测到。做数据清理的时候，对缺失值做分析是很重要的，我们要确定是否是数据收集的问题，或者缺失值是否会带来潜在的偏见。

内建的Python None值也被当做NA：

In [5]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

这里有一些用来处理缺失值的函数：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/zq0q8.png)

### 1. Filtering Out Missing Data

有一些方法来过滤缺失值。可以使用pandas.isnull和boolean indexing, 配合使用dropna。对于series，只会返回non-null数据和index values:

In [6]:
from numpy import nan as NA

In [7]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [8]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

上面的等同于：

In [9]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

对于DataFrame，会复杂一些。你可能想要删除包含有NA的row和column。dropna默认会删除包含有缺失值的row：

In [10]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


设定how=all只会删除那些全是NA的行：

In [12]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


删除列也一样，设置axis=1:

In [13]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [14]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


一种删除DataFrame row的相关应用是是time series data。假设你想要保留有特定数字的观测结果，可以使用thresh参数：

In [15]:
df = pd.DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,1.269236,-0.002011,-0.601553
1,1.791062,-0.955614,-2.151298
2,-1.344681,1.966870,-0.396087
3,-0.104675,-0.872303,-1.833648
4,-1.665642,1.272853,-0.043332
5,-0.810357,-0.339085,-0.066770
6,-0.717079,-0.218934,0.978064


In [16]:
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,1.269236,NaN,NaN
1,1.791062,NaN,NaN
2,-1.344681,NaN,-0.396087
3,-0.104675,NaN,-1.833648
4,-1.665642,1.272853,-0.043332
5,-0.810357,-0.339085,-0.066770
6,-0.717079,-0.218934,0.978064


In [17]:
df.dropna()

,0,1,2
4,-1.665642,1.272853,-0.043332
5,-0.810357,-0.339085,-0.066770
6,-0.717079,-0.218934,0.978064


In [18]:
df.dropna(thresh=2)

,0,1,2
2,-1.344681,NaN,-0.396087
3,-0.104675,NaN,-1.833648
4,-1.665642,1.272853,-0.043332
5,-0.810357,-0.339085,-0.066770
6,-0.717079,-0.218934,0.978064


### 2. Filling In Missing Data

若不删除缺失值，而是用一些数字填补。对于大部分目的，fillna是可以用的。调用fillna的时候设置好一个常用用来替换缺失值：

In [19]:
df.fillna(0)

,0,1,2
0,1.269236,0.000000,0.000000
1,1.791062,0.000000,0.000000
2,-1.344681,0.000000,-0.396087
3,-0.104675,0.000000,-1.833648
4,-1.665642,1.272853,-0.043332
5,-0.810357,-0.339085,-0.066770
6,-0.717079,-0.218934,0.978064


给fillna传入一个dict，可以给不同列替换不同的值：

In [20]:
df.fillna({1:0.5, 2:0})

,0,1,2
0,1.269236,0.500000,0.000000
1,1.791062,0.500000,0.000000
2,-1.344681,0.500000,-0.396087
3,-0.104675,0.500000,-1.833648
4,-1.665642,1.272853,-0.043332
5,-0.810357,-0.339085,-0.066770
6,-0.717079,-0.218934,0.978064


fillna返回一个新对象，但你可以使用in-place来直接更改原有的数据：

In [21]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,1.269236,0.000000,0.000000
1,1.791062,0.000000,0.000000
2,-1.344681,0.000000,-0.396087
3,-0.104675,0.000000,-1.833648
4,-1.665642,1.272853,-0.043332
5,-0.810357,-0.339085,-0.066770
6,-0.717079,-0.218934,0.978064


用于reindexing插入法在使用fillna的时候同样能使用：

In [22]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA

df

,0,1,2
0,0.515282,-1.428375,-0.008106
1,-1.986529,0.166937,1.099432
2,1.253719,NaN,-1.090445
3,-3.507374,NaN,0.067888
4,0.390261,NaN,NaN
5,0.970131,NaN,NaN


In [23]:
df.fillna(method='ffill')

,0,1,2
0,0.515282,-1.428375,-0.008106
1,-1.986529,0.166937,1.099432
2,1.253719,0.166937,-1.090445
3,-3.507374,0.166937,0.067888
4,0.390261,0.166937,0.067888
5,0.970131,0.166937,0.067888


In [24]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.515282,-1.428375,-0.008106
1,-1.986529,0.166937,1.099432
2,1.253719,0.166937,-1.090445
3,-3.507374,0.166937,0.067888
4,0.390261,NaN,0.067888
5,0.970131,NaN,0.067888


使用fillna可以我们做一些颇有创造力的事情。比如，可以传入一个series的平均值或中位数：

In [25]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

下面是fillna的一些参数：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/vtzrf.png)

## 7.2 Data Transformation（数据变换）

### 1. 删除重复值

In [26]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})

data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


DataFrame方法duplicated返回的是一个boolean Series，表示一个row是否是重复的（根据前一行来判断）：

In [34]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

drop_duplicateds返回一个DataFrame，会删除重复的部分：

In [35]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


上面两种方法都默认考虑所有列；另外，我们可以指定一部分来检测重复值。假设我们只想检测'k1'列的重复值：

In [36]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [37]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


duplicated和drop_duplicated默认保留第一次观测到的数值组合。设置keep='last'能返回最后一个：

In [38]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 2. Transforming Data Using a Function or Mapping

有时候我们可能希望做一些数据转换。比如下面一个例子，有不同种类的肉：

In [39]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


假设你想加一列，表明每种肉来源的动物是什么。我们可以写一个映射：

In [40]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

用于series的map方法接受一个函数，或是一个字典，包含着映射关系，但这里有一个小问题，有些肉是大写，有些是小写。因此，我们先用str.lower把所有的值变为小写:

In [42]:
lowercased = data["food"].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [43]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


我们也可以用一个函数解决上面的问题：

In [44]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

使用map是一个很简便的方法，用于element-wise转换和其他一些数据清洗操作。

### 3. Replacing Values

其实fillna是一个特殊换的替换操作。map可以用于修改一个object里的部分值，但是replace能提供一个更简单和更灵活的方法做到这点。下面是一个series：

In [45]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

这里-999可能是用来表示缺失值的标识符。用NA来替代的话，用replace，会产生一个新series（除非使用inplace=True）:

In [46]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

如果想要一次替换多个值，直接用一个list即可：

In [47]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

对于不同的值用不同的替换值，也是导入一个list：

In [48]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

参数也可以是一个dict：

In [49]:
data.replace({-999:np.nan, -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

注意：data.replace方法和data.str.replace方法是不同的，后者会对string进行element-wise替换。

### 4. Renaming Axis Indexes

像是series里的value一样，axis label也能类似地用函数或映射来转换，产生一个新的object。当然也可以设置in-place不产生新的数据：

In [50]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


与series相同，axis index有一个map方法：

In [51]:
transform = lambda x: x[:4].upper()
transform

<function __main__.<lambda>>

In [52]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

可以赋值给index，以in-place的方式修改DataFrame：

In [53]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


如果你想要创建一个转换后的版本，而且不用修改原始的数据，可以用rename:

In [54]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


注意，rename能用于dict一样的oject，

In [55]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'pekaboo'})

,one,two,pekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


rename能让你避免陷入手动赋值给index和columns的杂务中。可以用inplace直接修改原始数据：

In [56]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 5. Discretization and Binning

连续型数据经常被离散化或分散成bins（分箱）来分析。假设你有一组数据，你想把人分到不同的年龄组里：

In [57]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

我们把这些分到四个bin里，19~25, 26~35, 36~60, >60。可以用pandas里的cut：

In [58]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

返回的是一个特殊的Categorical object。我们看到的结果描述了pandas.cut如何得到bins。可以看作是一个string数组用来表示bin的名字，它内部包含了一个categories数组，用来记录不同类别的名字，并伴有表示ages的label（可以通过codes属性查看）：

In [59]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [63]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [67]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

这里pd.value_counts(cats)是pandas.cut后bin的数量。

这里我们注意一下区间。括号表示不包含，方括号表示包含。你可以自己设定哪一边关闭（right=False）:

In [68]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

你也可以用一个list或数组给labels选项来设定bin的名字：

In [73]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

如果你只是给一个bins的数量来cut，而不是自己设定每个bind的范围，cut会根据最大值和最小值来计算等长的bins。比如下面我们想要做一个标准正态分布的四个bins：

In [74]:
data = np.random.rand(20)

pd.cut(data, 4, precision=2)

[(0.76, 1.0], (0.76, 1.0], (0.28, 0.52], (0.28, 0.52], (0.28, 0.52], ..., (0.28, 0.52], (0.28, 0.52], (0.036, 0.28], (0.28, 0.52], (0.28, 0.52]]
Length: 20
Categories (4, interval[float64]): [(0.036, 0.28] < (0.28, 0.52] < (0.52, 0.76] < (0.76, 1.0]]

precision=2选项表示精确到小数点后两位。

一个近似的函数，qcut，会按照数据的分位数来分箱。取决于数据的分布，用cut通常不能保证每一个bin有一个相同数量的数据点。而qcut是按百分比来切的，所以可以得到等数量的bins：

In [75]:
data = np.random.randn(1000) 

cats = pd.qcut(data, 4)    # Cut into quartiles
cats

[(0.0177, 0.736], (0.0177, 0.736], (0.0177, 0.736], (-0.628, 0.0177], (0.736, 3.526], ..., (-3.056, -0.628], (-0.628, 0.0177], (0.0177, 0.736], (0.736, 3.526], (-0.628, 0.0177]]
Length: 1000
Categories (4, interval[float64]): [(-3.056, -0.628] < (-0.628, 0.0177] < (0.0177, 0.736] < (0.736, 3.526]]

In [76]:
pd.value_counts(cats)

(0.736, 3.526]      250
(0.0177, 0.736]     250
(-0.628, 0.0177]    250
(-3.056, -0.628]    250
dtype: int64

类似的，在cut中我们可以自己指定百分比：

In [77]:
cats2 = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
cats2

[(0.0177, 1.335], (0.0177, 1.335], (0.0177, 1.335], (-1.251, 0.0177], (1.335, 3.526], ..., (-3.056, -1.251], (-1.251, 0.0177], (0.0177, 1.335], (0.0177, 1.335], (-1.251, 0.0177]]
Length: 1000
Categories (4, interval[float64]): [(-3.056, -1.251] < (-1.251, 0.0177] < (0.0177, 1.335] < (1.335, 3.526]]

In [79]:
pd.value_counts(cats2,sort=False)

(-3.056, -1.251]    100
(-1.251, 0.0177]    400
(0.0177, 1.335]     400
(1.335, 3.526]      100
dtype: int64

在之后的章节我们还会用到cut和qcut，这些离散函数对于量化和群聚分析很有用。

### 6. Detecting and Filtering Outliers

过滤或转换异常值是数组操作的一个重头戏。下面的DataFrame有正态分布的数据：

In [81]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.head()

,0,1,2,3
0,-0.872743,1.977838,-0.837180,-0.038896
1,0.707862,-1.802960,-1.072872,2.029606
2,0.646305,0.435352,1.537727,-0.191223
3,0.342857,1.243595,0.032716,-0.751239
4,-0.084072,-0.119996,-1.315470,-1.740842


In [82]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.027278,-0.033319,0.014517,-0.006383
std,1.010676,0.990180,0.993463,0.976176
min,-3.008150,-2.905554,-3.118865,-3.111158
25%,-0.652208,-0.709131,-0.619980,-0.655762
50%,-0.020842,-0.009633,-0.022634,-0.013006
75%,0.671364,0.621744,0.670494,0.707397
max,3.740911,3.564471,3.464596,2.994585


假设我们想要找一个列中，绝对值大于3的数字：

In [83]:
col = data[2]
col[np.abs(col) > 3]

250    3.464596
313    3.164439
501    3.137253
593   -3.118865
Name: 2, dtype: float64

选中所有绝对值大于3的行，可以用any方法在一个boolean DataFrame上：

In [84]:
data[np.abs(data) > 3].head()

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [88]:
data[(np.abs(data) > 3).any(axis=1)]

,0,1,2,3
250,-1.304035,0.055204,3.464596,1.117414
294,-3.008150,0.262211,0.291699,-0.243769
313,1.184307,-0.171459,3.164439,0.478968
403,1.614297,0.402782,-0.364743,-3.111158
404,3.299609,-0.420842,-0.524577,1.819750
501,-0.938834,1.770992,3.137253,-0.217031
593,-0.700690,0.771875,-3.118865,1.028529
625,3.168810,0.191669,-0.739415,0.809506
626,0.994204,3.226087,0.830048,2.137377
648,3.740911,-1.425297,0.425976,-0.761696


下面是把绝对值大于3的数字直接变成-3或3：

In [91]:
data[np.abs(data) > 3] = np.sign(data) * 3
data[249:251]

,0,1,2,3
249,0.463531,0.090376,-1.224063,0.529964
250,-1.304035,0.055204,3.000000,1.117414


In [92]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.025712,-0.034110,0.013870,-0.006252
std,1.005607,0.987565,0.990653,0.975766
min,-3.000000,-2.905554,-3.000000,-3.000000
25%,-0.652208,-0.709131,-0.619980,-0.655762
50%,-0.020842,-0.009633,-0.022634,-0.013006
75%,0.671364,0.621744,0.670494,0.707397
max,3.000000,3.000000,3.000000,2.994585


np.sign(data)会根据值的正负号来得到1或-1："

In [93]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,-1.0
1,1.0,-1.0,-1.0,1.0
2,1.0,1.0,1.0,-1.0
3,1.0,1.0,1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0


### 7. Permutation and Random Sampling

排列（随机排序）一个series或DataFrame中的row，用numpy.random.permutation函数很容易就能做到。调用permutation的时候设定好你想要进行排列的axis，会产生一个整数数组表示新的顺序：

In [2]:
df = pd.DataFrame(np.arange(5*4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [3]:
sampler = np.random.permutation(5)
sampler

array([3, 4, 0, 2, 1])

这个数组能被用在基于iloc上的indexing或take函数：

In [4]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
4,16,17,18,19
0,0,1,2,3
2,8,9,10,11
1,4,5,6,7


为了选中一个随机的子集，而且没有代替功能(既不影响原来的值，返回一个新的series或DataFrame)，可以用sample方法：

In [5]:
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
1,4,5,6,7


如果想要生成的样本带有替代功能(即允许重复)，给sample中设定replace=True:

In [6]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

2   -1
3    6
4    4
0    5
3    6
2   -1
3    6
0    5
3    6
4    4
dtype: int64

### 8. Computing Indicator/Dummy Variables

> Dummy Variables：虚拟变量，又称虚设变量、名义变量或哑变量,用以反映值的属性的一个人工变量,是量化了的自变量,通常取值为0或1。

另一种在统计模型上的转换或机器学习应用是把一个categorical variable(类别变量)变为一个dummy or indicator matrix（虚拟或指示器矩阵）。如果DataFrame中的一列有k个不同的值，我们可以用k列1或0的矩阵或DataFrame表示。pandas有一个get_dummies函数实现这个工作，当然，你自己设计一个其实也不难。这里举个例子：

In [8]:
df = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'b'],
                  'data1':range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [9]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


在一些情况里，如果我们想要给column加一个prefix， 可以用data.get_dummies里的prefix参数来实现：

In [15]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


如果DataFrame中的a row中有多个类别，事情会变得复杂一些。我们来看一下MoviesLens 1M 数据集：

In [17]:
mnames = ['movie_id', 'title', 'genres']

movies = pd.read_table("../Data/Datasets/movielens/movies.dat", sep="::", 
                       header=None, names=mnames, engine='python')
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


给每个genre添加一个指示变量比较麻烦。首先我们先取出所有不同的类别：

In [18]:
all_genres = []

for x in movies.genres:
    all_genres.extend(x.split("|"))

genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

一种构建indicator dataframe的方法是先构建一个全是0的DataFrame：

In [22]:
zero_matrix = np.zeros((len(movies), len(genres)))

zero_matrix.shape

(3883, 18)

In [23]:
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


然后迭代每一部movie，并设置每一行中的dummies为1。使用dummies.columns来计算每一列的genre的指示器：

In [24]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [25]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

然后，使用.iloc，根据索引来设定值：

In [26]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split("|"))
    dummies.iloc[i, indices] = 1

dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


然后，我们可以与movies合并:

In [27]:
movies_windic = movies.join(dummies.add_prefix('Gere_'))
movies_windic.head()

,movie_id,title,genres,Gere_Animation,Gere_Children's,Gere_Comedy,Gere_Adventure,Gere_Fantasy,Gere_Romance,Gere_Drama,...,Gere_Crime,Gere_Thriller,Gere_Horror,Gere_Sci-Fi,Gere_Documentary,Gere_War,Gere_Musical,Gere_Mystery,Gere_Film-Noir,Gere_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


对于一个很大的数据集，这种构建多个成员指示变量的方法并不会加快速度。写一个低层级的函数来直接写一个numpy array，并把写过整合到DataFrame会更快一些。


一个有用的recipe诀窍是把get_dummies和离散函数（比如cut）结合起来：

In [28]:
np.random.seed(12345)

values = np.random.rand(10)
values

array([ 0.92961609,  0.31637555,  0.18391881,  0.20456028,  0.56772503,
        0.5955447 ,  0.96451452,  0.6531771 ,  0.74890664,  0.65356987])

In [30]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.cut(values, bins)

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

In [31]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0
